In [20]:
import numpy as np
import skimage as sk
from glob import glob
import napari
from cellpose import io, models, train
import os


In [21]:
img_files = sorted(glob("../Microglia_CellPose_Model/Images/*.tif"))
mask_files = sorted(glob("../Microglia_CellPose_Model/Masks/*.tif"))
imgs = list(map(sk.io.imread, img_files))
masks = list(map(sk.io.imread, mask_files))

In [ ]:
#os.mkdir('../Microglia_CellPose_Model/Training') #add in if directory does not exist yet
save = '../Microglia_CellPose_Model/Training'
for i in range(len(imgs)):
    split_img = np.split(imgs[i], imgs[i].shape[0],axis=0)
    split_mask = np.split(masks[i], masks[i].shape[0],axis=0)
    for c in range(len(split_img)):
        if np.max(split_mask[c]) == 0:
            continue
        else:
            labels = sk.measure.label(split_mask[c] > 0)
            sk.io.imsave(os.path.join(save,"Img_0"+str(i)+"_"+str(c)+".tif"),split_img[c],check_contrast=False)
            sk.io.imsave(os.path.join(save,"Img_0"+str(i)+"_"+str(c)+"_mask.tif"),labels,check_contrast=False)

In [24]:
help(io.load_train_test_data)

Help on function load_train_test_data in module cellpose.io:

load_train_test_data(train_dir, test_dir=None, image_filter=None, mask_filter='_masks', look_one_level_down=False)
    Loads training and testing data for a Cellpose model.
    
    Args:
        train_dir (str): The directory path containing the training data.
        test_dir (str, optional): The directory path containing the testing data. Defaults to None.
        image_filter (str, optional): The filter for selecting image files. Defaults to None.
        mask_filter (str, optional): The filter for selecting mask files. Defaults to "_masks".
        look_one_level_down (bool, optional): Whether to look for data in subdirectories of train_dir and test_dir. Defaults to False.
    
    Returns:
        images, labels, image_names, test_images, test_labels, test_image_names



In [29]:
train_dir = 'E:/ParadaKusz_Lab/Microglia_CellPose_Model/Training/'
test_dir = 'E:/ParadaKusz_Lab/Microglia_CellPose_Model/Test/'
io.logger_setup()

output = io.load_train_test_data(train_dir, test_dir,
                                mask_filter="_mask", look_one_level_down=False)
images, labels, image_names, test_images, test_labels, image_names_test = output

model = models.CellposeModel(model_type="cyto3",gpu=True)

model_path, train_losses, test_losses = train.train_seg(model.net,
                            train_data=images, train_labels=labels,
                            test_data=test_images, test_labels=test_labels,
                            weight_decay=0.1, learning_rate=1e-5,
                            n_epochs=100, model_name="Test")

2025-06-24 18:20:33,479 [INFO] WRITING LOG OUTPUT TO C:\Users\kristin.gallik\.cellpose\run.log
2025-06-24 18:20:33,481 [INFO] 
cellpose version: 	3.1.1.2 
platform:       	win32 
python version: 	3.10.16 
torch version:  	2.7.0+cu118
2025-06-24 18:20:33,546 [INFO] not all flows are present, running flow generation for all images
2025-06-24 18:20:36,745 [INFO] 139 / 139 images in E:/ParadaKusz_Lab/Microglia_CellPose_Model/Training/ folder have labels
2025-06-24 18:20:36,751 [INFO] not all flows are present, running flow generation for all images
2025-06-24 18:20:36,972 [INFO] 11 / 11 images in E:/ParadaKusz_Lab/Microglia_CellPose_Model/Test/ folder have labels
2025-06-24 18:20:36,974 [INFO] >> cyto3 << model set to be used
2025-06-24 18:20:37,094 [INFO] ** TORCH CUDA version installed and working. **
2025-06-24 18:20:37,096 [INFO] >>>> using GPU (CUDA)
2025-06-24 18:20:37,193 [INFO] >>>> loading model C:\Users\kristin.gallik\.cellpose\models\cyto3
2025-06-24 18:20:37,373 [INFO] >>>> mod

100%|██████████| 139/139 [02:43<00:00,  1.18s/it]

2025-06-24 18:23:22,777 [INFO] computing flows for labels



100%|██████████| 11/11 [00:03<00:00,  2.99it/s]

2025-06-24 18:23:26,571 [INFO] >>> computing diameters



100%|██████████| 11/11 [00:00<00:00, 49.16it/s]

2025-06-24 18:23:29,623 [WARNING] 107 train images with number of masks less than min_train_masks (5), removing from train set


2025-06-24 18:23:29,953 [INFO] >>> normalizing {'lowhigh': None, 'percentile': None, 'normalize': True, 'norm3D': True, 'sharpen_radius': 0, 'smooth_radius': 0, 'tile_norm_blocksize': 0, 'tile_norm_smooth3D': 1, 'invert': False}
2025-06-24 18:23:34,014 [INFO] >>> n_epochs=100, n_train=32, n_test=11
2025-06-24 18:23:34,016 [INFO] >>> AdamW, learning_rate=0.00001, weight_decay=0.10000
2025-06-24 18:23:36,842 [INFO] >>> saving model to e:\ParadaKusz_Lab\OIC-136_Microglia\models\Test


RuntimeError: running_mean should contain 1 elements not 2